In [1]:
import os
import sys
import math
import datetime

mainDir = os.path.dirname(os.path.abspath(""))
sys.path.append(mainDir)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import QuantLib as ql

from matplotlib.ticker import PercentFormatter

from ir import *

A QuantLib-Python companion for the Ametrano-Bianchetti paper:
https://www.implementingquantlib.com/2023/09/ametrano-bianchetti.html

In [3]:
eoniaDataframe = pd.read_csv(
    os.path.abspath(os.path.join(mainDir, "data/eonia11122012.csv")),
    sep=';'
)
eoniaDataframe.head(10)

,Bootstrapping Instrument,"Rate, %",Start Date,End Date
0,EUR_YC_OND,0.040,Tue 11 Dec 2012,Wed 12 Dec 2012
1,EUR_YC_TND,0.040,Wed 12 Dec 2012,Thu 13 Dec 2012
2,EUR_YC_SND,0.040,Thu 13 Dec 2012,Fri 14 Dec 2012
3,EUR_YC_EONSW,0.070,Thu 13 Dec 2012,Thu 20 Dec 2012
4,EUR_YC_EON2W,0.069,Thu 13 Dec 2012,Thu 27 Dec 2012
5,EUR_YC_EON3W,0.078,Thu 13 Dec 2012,Thu 03 Jan 2013
6,EUR_YC_EON1M,0.074,Thu 13 Dec 2012,Mon 14 Jan 2013
7,EUR_YC_EONECBJan13,0.046,Wed 16 Jan 2013,Wed 13 Feb 2013
8,EUR_YC_EONECBFEB13,0.016,Wed 13 Feb 2013,Wed 13 Mar 2013
9,EUR_YC_EONECBMAR13,-0.007,Wed 13 Mar 2013,Wed 10 Apr 2013


In [4]:
eoniaDataframe['StartDate'] = eoniaDataframe['Start Date'].apply(
    lambda x: datetime.datetime.strptime(x, '%a %d %b %Y').date()
)

eoniaDataframe['EndDate'] = eoniaDataframe['End Date'].apply(
    lambda x: datetime.datetime.strptime(x, '%a %d %b %Y').date()
)

eoniaDataframe['Rate'] = eoniaDataframe['Rate, %'] / 100

eoniaDataframe.head()

,Bootstrapping Instrument,"Rate, %",Start Date,End Date,StartDate,EndDate,Rate
0,EUR_YC_OND,0.040,Tue 11 Dec 2012,Wed 12 Dec 2012,2012-12-11,2012-12-12,0.00040
1,EUR_YC_TND,0.040,Wed 12 Dec 2012,Thu 13 Dec 2012,2012-12-12,2012-12-13,0.00040
2,EUR_YC_SND,0.040,Thu 13 Dec 2012,Fri 14 Dec 2012,2012-12-13,2012-12-14,0.00040
3,EUR_YC_EONSW,0.070,Thu 13 Dec 2012,Thu 20 Dec 2012,2012-12-13,2012-12-20,0.00070
4,EUR_YC_EON2W,0.069,Thu 13 Dec 2012,Thu 27 Dec 2012,2012-12-13,2012-12-27,0.00069


# QuantLib case

In [6]:
today = ql.Date(11, ql.December, 2012)
ql.Settings.instance().evaluationDate = today

helpers = [
    ql.DepositRateHelper(
        ql.QuoteHandle(ql.SimpleQuote(rate / 100)),
        ql.Period(1, ql.Days), # tenor
        fixingDays,
        ql.TARGET(),
        ql.Following,
        False,
        ql.Actual360(),
    )
    for rate, fixingDays in [(0.04, 0), (0.04, 1), (0.04, 2)]
]
eonia = ql.Eonia()
helpers += [
    ql.OISRateHelper(
        2, ql.Period(*tenor), ql.QuoteHandle(ql.SimpleQuote(rate / 100)), eonia
    )
    for rate, tenor in [
        (0.070, (1, ql.Weeks)),
        (0.069, (2, ql.Weeks)),
        (0.078, (3, ql.Weeks)),
        (0.074, (1, ql.Months)),
    ]
]
helpers += [
    ql.DatedOISRateHelper(
        start_date, end_date, ql.QuoteHandle(ql.SimpleQuote(rate / 100)), eonia
    )
    for rate, start_date, end_date in [
        (0.046, ql.Date(16, ql.January, 2013), ql.Date(13, ql.February, 2013)),
        (0.016, ql.Date(13, ql.February, 2013), ql.Date(13, ql.March, 2013)),
        (-0.007, ql.Date(13, ql.March, 2013), ql.Date(10, ql.April, 2013)),
        (-0.013, ql.Date(10, ql.April, 2013), ql.Date(8, ql.May, 2013)),
        (-0.014, ql.Date(8, ql.May, 2013), ql.Date(12, ql.June, 2013)),
    ]
]
eonia_curve_c = ql.PiecewiseLogCubicDiscount(
    0, ql.TARGET(), helpers, ql.Actual365Fixed()
    # 0, ql.TARGET(), helpers, ql.Actual360()
)
eonia_curve_c.enableExtrapolation()
print(list(eonia_curve_c.dates()))
[eonia_curve_c.discount(_date) for _date in list(eonia_curve_c.dates())]

[Date(11,12,2012), Date(12,12,2012), Date(13,12,2012), Date(14,12,2012), Date(20,12,2012), Date(27,12,2012), Date(3,1,2013), Date(14,1,2013), Date(13,2,2013), Date(13,3,2013), Date(10,4,2013), Date(8,5,2013), Date(12,6,2013)]


[1.0,
 0.9999988888901234,
 0.9999977777814815,
 0.999996666674074,
 0.9999841668858677,
 0.9999709452277842,
 0.9999522799527308,
 0.9999320044764968,
 0.9998932528635275,
 0.9998808099015456,
 0.9998862537258497,
 0.9998963637881604,
 0.9999099736739131]

# Own

In [8]:
# eoniaDataframe.head(7)

In [9]:
todayDate = eoniaDataframe.StartDate[0]
spotDate = todayDate + Period('2D')

stubPeriod = ShortBack()
calendar = TargetCalendar()
businessDayConvention = Following()

dates = [todayDate]

# deposits
dates += [
    calendar.advance(
        date=todayDate,
        period=Period(tenor),
        businessDayConvention=businessDayConvention,
        endOfMonth=False
    )
    for tenor in ['1D', '2D', '3D']
]

# ois to 1M
dates += [
    calendar.advance(
        date=spotDate,
        period=Period(tenor),
        businessDayConvention=businessDayConvention,
        endOfMonth=False
    )
    for tenor in ['1W', '2W', '3W', '1M']
]

dates += [eoniaDataframe["StartDate"][7]] 
dates += eoniaDataframe["EndDate"][7:12].tolist()

initialNodes = {_date: 1 for _date in dates}
initialCurve = DiscountCurve(
    dates=list(initialNodes.keys()),
    discountFactors=list(initialNodes.values()),
    dayCounter=Act365Fixed()
    # dayCounter=Act360()
)


In [35]:
# deposits
swaps = [
    InterestRateSwap(
        curve=initialCurve,
        fixedRate=rate,
        effectiveDate=effectiveDate,
        terminationDate=terminationDate,
        fixFrequency='1D',
        floatFrequency='1D',
        endOfMonth=False,
        businessDayConvention=businessDayConvention,
        dayCounter=Act360(),
        stubPeriod=stubPeriod,
        calendar=calendar,
        notional=1.
    )
    for rate, effectiveDate, terminationDate in zip(
        eoniaDataframe.Rate[:3].tolist(),
        dates[:3],
        dates[1:4]
    )
]

# from 1W to 1M OIS
swaps += [
    InterestRateSwap(
        curve=initialCurve,
        fixedRate=rate,
        effectiveDate=spotDate,
        terminationDate=terminationDate,
        fixFrequency=tenor,
        floatFrequency=tenor,
        endOfMonth=False, # TODO True or False for 1M?
        businessDayConvention=businessDayConvention,
        dayCounter=Act365Fixed(),
        stubPeriod=stubPeriod,
        calendar=calendar,
        notional=1.
    )
    for rate, tenor, terminationDate in zip(
        eoniaDataframe.Rate[3:7].tolist(),
        ['1W', '2W', '3W', '1M'],
        dates[4:8]
    )
]

swaps += [
    InterestRateSwap(
        curve=initialCurve,
        fixedRate=rate,
        effectiveDate=spotDate,
        terminationDate=terminationDate,
        fixFrequency=tenor,
        floatFrequency=tenor,
        endOfMonth=False, # TODO True or False for 1M?
        businessDayConvention=businessDayConvention,
        dayCounter=Act365Fixed(),
        stubPeriod=stubPeriod,
        calendar=calendar,
        notional=1.
    )
    for rate, tenor, terminationDate in zip(
        eoniaDataframe.Rate[3:7].tolist(),
        ['1W', '2W', '3W', '1M'],
        dates[4:8]
    )
]

swaps += [
    InterestRateSwap(
        curve=initialCurve,
        fixedRate=rate,
        effectiveDate=startDate,
        terminationDate=endDate,
        fixFrequency='1M',
        floatFrequency='1M',
        endOfMonth=True, # TODO True or False for 1M?
        businessDayConvention=businessDayConvention,
        dayCounter=Act365Fixed(),
        stubPeriod=stubPeriod,
        calendar=calendar,
        notional=1.
    )
    for startDate, endDate, rate in zip(
        eoniaDataframe["StartDate"][7:12],
        eoniaDataframe["EndDate"][7:12],
        eoniaDataframe["Rate"][7:12]
    )
]

curve, convergenceStatus = CurveBootstrapping(
    initialGuessNodes=initialNodes,
    swaps=swaps,
    # dayCounter=Act365Fixed(),
    dayCounter=Act360(),
    curveInterpolator=LogLinearInterpolator
).solve()



In [37]:
for _date in dates:
    print(f"{_date}: {curve.getDiscountFactor(_date).realPart}")

2012-12-11: 1.0
2012-12-12: 0.9999988888901234
2012-12-13: 0.9999977777814815
2012-12-14: 0.999996666674074
2012-12-20: 0.9999841668858764
2012-12-27: 0.9999709452277845
2013-01-03: 0.9999522799527436
2013-01-14: 0.9999320044762982
2013-01-16: 1.0000190538524134
2013-02-13: 0.9999832766729592
2013-03-13: 0.999970832591487
2013-04-10: 0.9999762769067725
2013-05-08: 0.9999863878802502
2013-06-12: 0.9999999989813659
